# Plot monthly time spent for a recurring event from a Google calendar

use the data manually exported from google calendar into ics, then manually converted to csv

ICS to CSV converter used: https://github.com/johnyluyte/ICS-to-Excel-CSV

In [ ]:
import pandas as pd
import numpy as np
import datetime
# import time

# import matplotlib.pyplot as plt
import plotly.graph_objects as go
# from plotly import tools
# import plotly.offline as py
# import plotly.express as px
from plotly.offline import iplot

# %matplotlib inline





In [ ]:
#import data
timetable = pd.read_csv("google_calendar.csv")
timetable.drop(["詳細","標題"],axis=1,inplace=True)
timetable.columns=("开始", "结束","标题")
#drop all other entries except for the EVENT
EVENTNAME = "event"
timetable = timetable.loc[lambda x: (timetable['标题'] == EVENTNAME), :] 
timetable = timetable.reset_index(drop=True)
timetable.head(10)

In [ ]:
# testing date time conversion
# print(time.mktime(datetime.datetime.strptime(timetable["开始"][8][:8], "%Y%m%d").timetuple()))
# print(datetime.datetime.strptime(timetable["开始"][8][:8]+timetable["开始"][8][-7:-1], "%Y%m%d%H%M%S"))

In [ ]:
timetable.info()

In [ ]:
#convert string into date
print(f'before:{timetable["开始"][0]}\n')
timetable["开始"] = timetable["开始"].apply(lambda x: datetime.datetime.strptime(x[:8]+x[-7:-1], "%Y%m%d%H%M%S").replace(tzinfo=datetime.timezone.utc).astimezone(tz=None))
print(f'after:{timetable["开始"][0]}\n')
timetable["结束"] = timetable["结束"].apply(lambda x: datetime.datetime.strptime(x[:8]+x[-7:-1], "%Y%m%d%H%M%S").replace(tzinfo=datetime.timezone.utc).astimezone(tz=None))
timetable.info()

In [ ]:
#calculate event duration
timetable["duration"] = None
timetable["duration"] = timetable.apply(lambda x: (x["结束"]-x["开始"]).seconds/60/60, axis=1)
#atomicity
timetable["day"] = None
timetable["day"] = timetable.apply(lambda x: x["结束"].date(), axis=1)#make yyyy-mm-dd
timetable["day"] = pd.to_datetime(timetable["day"])#convert string to date
timetable["month"] = None
timetable["month"] = timetable.apply(lambda x: x["结束"].month, axis=1)
timetable["year"] = None
timetable["year"] = timetable.apply(lambda x: x["结束"].year, axis=1)
timetable.info()
timetable.head()

### make a table for the monthly plot

In [ ]:
timetable2 = timetable[["day","month","year","标题","duration"]].copy()
timetable2 = timetable2.value_counts().reset_index(name='counts').sort_values("day")
timetable2 = timetable2.reset_index(drop=True)
timetable2.tail()

In [ ]:
#calculate total hours per day
timetable2["hours_per_day"] = None
timetable2["hours_per_day"] = timetable2.apply(lambda x: x["duration"]*x["counts"], axis=1)
timetable2 = timetable2.reset_index(drop=True)
timetable2 = timetable2.sort_values(by="day")
timetable2.tail()

In [ ]:
#check how the result of the query for a particular month looks like
timetable2.loc[lambda x: x["month"]==3,:]

### plot monthly total

In [ ]:
#plot using pandas function
YEAR_TO_REPORT = 2021  #change this

timetable3_monthly = {}
months_in_the_year = timetable2.loc[lambda x: (timetable2['year'] == YEAR_TO_REPORT), :]["month"].unique()
for i in range(months_in_the_year[0],months_in_the_year[-1]+1): #needed +1 bc range() doesnt include the last item
    timetable3_monthly[i] = timetable2.loc[lambda x: (timetable2['month'] == i)&(timetable2['year'] == YEAR_TO_REPORT), :]["hours_per_day"].sum()
pd.Series(timetable3_monthly).plot(kind="bar", title=f"hours of {timetable2['标题'][0]} per month in {YEAR_TO_REPORT}", ylabel="hours", xlabel="months", stacked=False, figsize=(10,5), colormap="winter")

In [ ]:
#print out totals
pd.Series(timetable3_monthly).rename(index=("小时")).to_frame()#.astype(int)

In [ ]:
#generate a full timeline with days for the chart, based off the table with all input data
#as it is also needed to show days without the event
date_range = None
date_range = pd.DataFrame(pd.date_range(start=timetable2['day'][:1].values[0],end=timetable2['day'][-1:].values[0]), columns=["day"])
date_range.info()

In [ ]:
#plot hourly data for a given month and year using plotly
#the graph is dynamic

YEAR_TO_REPORT = 2021  #change this
MONTH_TO_REPORT = 7  #change this

#put the hourly data onto the timeline
events_month = timetable2.merge(date_range, on="day", how="right")

for index, row in events_month.iterrows():
    if events_month["day"][index].month != MONTH_TO_REPORT or events_month["day"][index].year != YEAR_TO_REPORT:
        events_month = events_month.drop(index)
events_month = events_month.reset_index(drop=True)

trace1 = go.Bar(x=events_month["day"],y=events_month["counts"], 
                text = events_month["counts"],opacity=0.75, textposition='auto')
data = [trace1]
layout = dict(title = f'{EVENTNAME}时间',
              xaxis= dict(title=f'{YEAR_TO_REPORT} year, {MONTH_TO_REPORT}th month, days',
                          ticklen= 5,zeroline= False, tickmode = 'linear',tickangle=-45)
             )
fig = dict(data = data, layout = layout)
iplot(fig)


#plot using pandas function
#the plot is static
events_month["day"] = events_month.apply(lambda x: x["day"].day, axis=1) #replace date with int day for plot axis ticks
events_month[["day", "duration","hours_per_day"]].plot(kind="bar", x="day",y="hours_per_day", title=f'{YEAR_TO_REPORT} year, {MONTH_TO_REPORT}th month', ylabel="hours", xlabel="days", stacked=False, figsize=(15,5), colormap="winter")



#calculate totals
hour_sum_total = 0
events_month = events_month.dropna()
for index, row in events_month.iterrows():
    hour_sum_total+=(events_month["duration"][index]*events_month["counts"][index])
print(f'total events: {int(events_month["counts"].sum())}')
print(f'total hours: {float(hour_sum_total):.1f}')  